In [105]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import svm
from decimal import *

read train,val and test from files

In [2]:
train = pd.read_csv('input/train.tsv', sep='\t')
test = pd.read_csv('input/test.tsv', sep='\t')

In [72]:
X = train.iloc[:100,1:151]

In [73]:
y = train.loc[:99,"target"]

In [97]:
len(y)

100

In [59]:
kfold = KFold(10, True, 1)

/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [107]:
for train_index , test_index in kfold.split(X):
    zero_features = []
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    mse,zero_features = (Lasso_finetuning(X_train,y_train,X_test,y_test))
    #lasso_dict[mse] = zero_features

    # drop zero corelation features
    X_train.drop(zero_features, axis = 1, inplace=True)
    X_test.drop(zero_features, axis = 1, inplace=True)

    #svm classifier
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy: ",Decimal(metrics.accuracy_score(y_test, y_pred)))

    #random forests classifier
    
    
    

/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(90, 25) (10, 25)
Accuracy:  0.6999999999999999555910790149937383830547332763671875


/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(90, 30) (10, 30)
Accuracy:  0.8000000000000000444089209850062616169452667236328125


/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(90, 31) (10, 31)
Accuracy:  0.8000000000000000444089209850062616169452667236328125


/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(90, 19) (10, 19)
Accuracy:  0.8000000000000000444089209850062616169452667236328125
(90, 31) (10, 31)
Accuracy:  0.59999999999999997779553950749686919152736663818359375


/Users/kondo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [51]:
lasso_dict

{0.09736291227690382: ['graph_dim_1',
  'graph_dim_2',
  'graph_dim_3',
  'graph_dim_5',
  'graph_dim_6',
  'graph_dim_7',
  'graph_dim_8',
  'graph_dim_9',
  'graph_dim_10',
  'graph_dim_11',
  'graph_dim_12',
  'graph_dim_13',
  'graph_dim_14',
  'graph_dim_15',
  'graph_dim_16',
  'graph_dim_17',
  'graph_dim_18',
  'graph_dim_19',
  'graph_dim_20',
  'graph_dim_21',
  'graph_dim_22',
  'graph_dim_23',
  'graph_dim_24',
  'graph_dim_26',
  'graph_dim_27',
  'graph_dim_28',
  'graph_dim_29',
  'graph_dim_30',
  'graph_dim_31',
  'graph_dim_32',
  'graph_dim_33',
  'graph_dim_36',
  'graph_dim_37',
  'graph_dim_38',
  'graph_dim_39',
  'graph_dim_40',
  'graph_dim_43',
  'graph_dim_44',
  'graph_dim_45',
  'graph_dim_46',
  'graph_dim_47',
  'graph_dim_48',
  'graph_dim_49',
  'graph_dim_50',
  'graph_dim_51',
  'graph_dim_52',
  'graph_dim_53',
  'graph_dim_54',
  'graph_dim_55',
  'graph_dim_56',
  'graph_dim_57',
  'graph_dim_58',
  'graph_dim_59',
  'graph_dim_60',
  'graph_dim_61

**Lasso_finetuning** : returns a pair of mse of train dataset and zero corelation features

In [99]:
def Lasso_finetuning(X_train, y_train,X_test,Y_test):
    lasso = Lasso(max_iter=10000, normalize=True)
    #alphas = [0.0001, 0.00001, 0.001, 0.01, 0.1]
    lassocv = LassoCV(alphas=None, cv=50, max_iter=100000, normalize=True)
    lassocv.fit(X_train, y_train)
    lasso.set_params(alpha=lassocv.alpha_)
    lasso.fit(X_train, y_train)
    mse = mean_squared_error(y_test, lasso.predict(X_test))

    #print("alpha : ",lassocv.alpha_)
    zero_features = []
    zero_count = 0;
    ser = pd.Series(lasso.coef_, index=X_train.columns)
    for i, v in ser.items():
        #print(i,v)
        if (abs(v) == 0):
            zero_count += 1
            zero_features.append(i)
    #print(zero_count,' zero corelation features')
    return mse,zero_features

**accuracy_score** : returns accuracy score of the model 

In [104]:
def accuracy_score(y_test,y_pred):
    count = np.sum(np.all(y_test == y_pred))
    print(count)
    return count / len(y_test)